Mostly following
- https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
- https://github.com/AKASHKADEL/dcgan-mnist

Imports

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms

Set device to GPU (cuda) if available

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Download mnist dataset

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))])

train_dataset = datasets.MNIST(
    root='data',
    train=True,
    transform=transform,
    download=False,
)

Print some datapoints using matplotlib

Set hyperparameter and load datasets into pytorch DataLoader. This allows to batch data and shuffle after each epoch. It also allows using multiple processors and load data directly into CUDA tensors.

In [ ]:
batch_size = 128
num_classes = 10
learning_rate = 0.002
num_epochs = 5
num_color_channels = 1
num_feature_maps_g = 32
num_feature_maps_d = 32
size_z = 100
adam_beta1 = 0.2
num_gpu = 0

dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

Models

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, num_feature_maps, num_color_channels):
        super(Discriminator, self).__init__()
        self.network = nn.Sequential(

            nn.Conv2d(num_color_channels, num_feature_maps, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(num_feature_maps, num_feature_maps * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_feature_maps * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(num_feature_maps * 2, num_feature_maps * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(num_feature_maps * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(num_feature_maps * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.network(input)
        return output.view(-1, 1).squeeze(1)

class Generator(nn.Module):
    def __init__(self, size_z, num_feature_maps, num_color_channels):
        super(Generator, self).__init__()
        self.network = nn.Sequential(
            nn.ConvTranspose2d(size_z, num_feature_maps * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(num_feature_maps * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(num_feature_maps*4, num_feature_maps * 2, 3, 2, 1, bias=False),
            nn.BatchNorm2d(num_feature_maps * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(num_feature_maps * 2, num_feature_maps, 4, 2, 1, bias=False),
            nn.BatchNorm2d(num_feature_maps),
            nn.ReLU(True),

            nn.ConvTranspose2d(num_feature_maps, num_color_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.network(input)
        return output

Load Generator (CNN) class

In [ ]:
generator = Generator(size_z=size_z,
                      num_feature_maps=num_feature_maps_g,
                      num_color_channels=num_color_channels).to(device)

Load Discriminator (CNN) class

In [ ]:
discriminator = Discriminator(num_feature_maps=num_feature_maps_d,
                              num_color_channels=num_color_channels).to(device)

loss function and optimization functions

In [ ]:
criterion = nn.BCELoss()

real_label = 1.
fake_label = 0.

optimizerG = optim.Adam(generator.parameters(), lr=learning_rate, betas=(adam_beta1, 0.999))
optimizerD = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(adam_beta1, 0.999))

Training function

In [ ]:
print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        # get batch-size from actual image batch
        bs = real_images.shape[0]

        # -- train discriminator --

        # reset/clear discriminators gradient
        discriminator.zero_grad()

        # move images to either CPU or GPU
        real_images = real_images.to(device)

        # creates a label tensor filled with 1s
        label = torch.full((bs,), real_label, device=device)

        # get probs for discriminators guess on the real images
        output = discriminator(real_images)

        # get loss for real images. that means it calculates the difference
        # between the output of the model with the current parameter and the
        # target (goal) of what the model is supposed to do
        # output --> current outcome of the model
        # label  --> target of the model
        lossD_real = criterion(output, label)

        # calculates the gradient (using chain-rule)
        # see https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html
        lossD_real.backward()

        # Gets the mean value of all results from the discriminator to get an average
        # probability of all sample evaluations (for real data ) --> D(x)
        D_x = output.mean().item()

        # create noise as an input for the G in order to create fake images
        noise = torch.randn(bs, size_z, 1, 1, device=device)

        # use generator to map input noise to an output that is supposed do become fake images during training
        fake_images = generator(noise)

        # creates a label tensor filled with 0s
        label.fill_(fake_label)

        # get discriminators guess on fake images
        output = discriminator(fake_images.detach())

        # get loss for fake images
        lossD_fake = criterion(output, label)

        # adjust parameter to identify fakes
        lossD_fake.backward()

        # gets the mean value of all results from the discriminator to get an average
        # probability of all sample evaluations. this time for the fake images that were
        # generated by the generator --> D(G(z))
        D_G_z1 = output.mean().item()

        # calculate loss
        lossD = lossD_real + lossD_fake

        # adjust models (discriminator) parameter
        optimizerD.step()

        # -- train generator --

        # reset/clear generators gradient
        generator.zero_grad()

        # creates a label tensor filled with 1s
        label.fill_(real_label)

        # get discriminators guess on fake images
        output = discriminator(fake_images)

        # get loss for fake images
        lossG = criterion(output, label)

        # adjust parameter to generate fakes
        lossG.backward()

        # gets the mean value of all results from the discriminator to get an average
        # probability of all sample evaluations. this time for the fake images that were
        # generated by the generator --> D(G(z))
        D_G_z2 = output.mean().item()

        # adjust models (generator) parameter
        optimizerG.step()

        if (i + 1) % 100 == 0:
            print(
                'Epoch [{}/{}], step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, Discriminator - D(G(x)): {:.2f}, Generator - D(G(x)): {:.2f}'
                .format(
                    epoch + 1,
                    num_epochs,
                    i + 1,
                    batch_size,
                    lossD.item(),
                    lossG.item(),
                    D_x,
                    D_G_z1,
                    D_G_z2
                )
            )
    generator.eval()
    generator.train()

Starting Training Loop...


KeyboardInterrupt: 

In [ ]:
torch.save(generator.state_dict(), 'model.ckpt')